# Setup

In [ ]:
!nvidia-smi
!pip install "ultralytics>=8.3.50" roboflow==1.*

In [ ]:
!rm -rf /content/sample_data/

# Load Dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="UGuKv3GDa6PNiTMeOoLd")
project = rf.workspace("workspaceendeavor360").project("formula-student-cones-roz6f") #personal workspace
version = project.version(1)
dataset = version.download("yolov12")
print("Dataset downloaded to:", dataset.location)


# To save after Disconnect

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/cone-detector-yolov12-1024"
!mkdir -p "$PROJECT_DIR"


# Now train on the custom exported dataset

In [ ]:
from ultralytics import YOLO
import os
os.chdir(dataset.location)

# Load base model
model = YOLO("yolov8n.pt")
# Start training
results = model.train(
    data="data.yaml",
    epochs=50,
    imgsz=1024,
    batch=8,
    device=0,
    rect=True,
    project=PROJECT_DIR,
    name="cones_yolov12_1024"
)


# Test

In [ ]:
os.chdir("/content")

%rm -rf /content/pred_custom

import os, cv2
from ultralytics import YOLO

best_path = "/content/drive/MyDrive/cone-detector-yolov12-1024/cones_yolov12_1024/weights/best.pt"
model = YOLO(best_path)

test_path = "/content/Formula-Student-Cones-1/test/images/"

results = model.predict(source=test_path, conf=0.5, save=False)

out_dir = "/content/pred_custom"
os.makedirs(out_dir, exist_ok=True)

for i, r in enumerate(results):
    img = r.plot(labels=True, conf=True, line_width=1, font_size=0.2)
    cv2.imwrite(os.path.join(out_dir, f"{i:04d}.jpg"), img)

print("Custom predictions saved to:", out_dir)


In [ ]:
os.chdir(dataset.location)
metrics = model.val(data="data.yaml", split="test", imgsz=1024, batch=8, device=0)
print(metrics)